In [1]:
from google.colab import drive

In [2]:
drive.mount("/content/drive/")

Mounted at /content/drive/


In [3]:
import os

os.chdir("/content/drive/My Drive/Colab_Notebooks/ANNClassification")

In [4]:
! pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [6]:
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


# Preprocess data


In [7]:
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


# Encoding Gender

In [8]:
label_encoder_gender = LabelEncoder()
data['Gender']= label_encoder_gender.fit_transform(data['Gender'])
data.tail()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1
9999,792,France,0,28,4,130142.79,1,1,0,38190.78,0


In [9]:
# one hot encode geography col
from sklearn.preprocessing import OneHotEncoder

one_hot_encoder =OneHotEncoder()
geo_encoder = one_hot_encoder.fit_transform(data[['Geography']])
geo_encoder

<10000x3 sparse matrix of type '<class 'numpy.float64'>'
	with 10000 stored elements in Compressed Sparse Row format>

In [10]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [11]:
one_hot_encoder.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [12]:
encoded_geo_df = pd.DataFrame(geo_encoder.toarray(), columns=one_hot_encoder.get_feature_names_out(['Geography']))

In [13]:
encoded_geo_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [14]:
# concatinate encoded df with data

data = data.drop('Geography', axis=1)
data = pd.concat([data, encoded_geo_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [15]:
data.groupby('Exited').count()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
Exited,,,,,,,,,,,,
0,7963,7963,7963,7963,7963,7963,7963,7963,7963,7963,7963,7963
1,2037,2037,2037,2037,2037,2037,2037,2037,2037,2037,2037,2037


In [16]:
## save the encoders and scaler

with open('label_encoder_gender.pkl', 'wb') as file:         # wb- write byte mode
  pickle.dump(label_encoder_gender,file)

with open('one_hot_encoder.pkl', 'wb') as file:
  pickle.dump(one_hot_encoder,file)

In [17]:
# divide data to dependent and independent feature

X= data.drop('Exited', axis=1)
y= data['Exited']

# splitting data to train and test
X_train, X_, y_train, y_ = train_test_split(X, y, test_size=0.2, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_, y_, test_size=0.5, random_state=42)

#scaling (normalizing) features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)     #--scaling using same param(u & std.deviation)
X_val = scaler.transform(X_val)




In [18]:
print(X_train.shape)
print(X_test.shape)
print(X_val.shape)

(8000, 12)
(1000, 12)
(1000, 12)


In [19]:
with open('scaler.pkl', 'wb') as file:
  pickle.dump(scaler,file)

# ANN Implimentation

In [20]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import datetime


In [21]:
(X_train.shape[1],)

(12,)

In [22]:
# building ann model

model = Sequential([
    tf.keras.Input(shape = (X_train.shape[1],)),
    Dense(units=64, activation='relu'),   # HL1 connected with i/p Layer
    Dense(32, activation='relu'),   # HL2
    Dense(1, activation='sigmoid')  # o/p layer
    ]
)



In [23]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │             832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [24]:
## compiling model
# as our data is imbalanced so not using accurancy metrics
from tensorflow.keras.metrics import Recall

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss=tf.keras.losses.BinaryCrossentropy(), metrics=["accuracy", Recall()])


In [25]:
## set up early stopping
from tensorflow.keras.callbacks import EarlyStopping

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [26]:
## train model
if len(tf.config.list_physical_devices('GPU')) ==0:
  D = '/cpu:0'
else:
  D = '/gpu:0'
with tf.device(D):
  history = model.fit(
      X_train, y_train,
      validation_data=(X_val, y_val),
      epochs=100,
      callbacks=early_stopping_callback
  )

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.8078 - loss: 0.4455 - recall: 0.2250 - val_accuracy: 0.8560 - val_loss: 0.3527 - val_recall: 0.5674
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8521 - loss: 0.3647 - recall: 0.4362 - val_accuracy: 0.8610 - val_loss: 0.3361 - val_recall: 0.4279
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8589 - loss: 0.3432 - recall: 0.4579 - val_accuracy: 0.8670 - val_loss: 0.3415 - val_recall: 0.5023
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8588 - loss: 0.3450 - recall: 0.4592 - val_accuracy: 0.8650 - val_loss: 0.3321 - val_recall: 0.4744
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8571 - loss: 0.3438 - recall: 0.4601 - val_accuracy: 0.8710 - val_loss: 0.3388 - val_recall: 0.5442
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8590 - loss: 0.3354 - recall: 0.4707 - val_accuracy: 0.8620 - val_loss: 0.3392 - val_recall: 0.446

In [27]:
from sklearn.metrics import confusion_matrix,recall_score,precision_score,f1_score

In [28]:
y_prob = model.predict(X_val)
y_pred = (y_prob > 0.5).astype(int)

Con_matrix = confusion_matrix(y_val, y_pred)

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


[[TN FP]
                          <br>[FN TP]]

In [29]:
print(Con_matrix)

[[751  34]
 [105 110]]


In [30]:
recall_score(y_val,y_pred), precision_score(y_val,y_pred), f1_score(y_val,y_pred)

(0.5116279069767442, 0.7638888888888888, 0.6128133704735376)

# ANN architecture

In [31]:
pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 11.7 MB/s eta 0:00:00


In [32]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.metrics import Recall
import tensorflow as tf
import keras_tuner as kt


In [33]:

# Define a function to create a model with given parameters
def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Choice('units',[16,32,64,128]),
                    activation='relu', input_shape=(X_train.shape[1],)))

    # Add variable number of layers
    for i in range(hp.Int('layers', 1, 3)):
      model.add(Dense(units=hp.Choice(f'layer_{i}_units', [16, 32, 64, 128]), activation='relu'))

    model.add(Dense(1, activation='sigmoid'))  # Output layer

    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', [0.001, 0.0001])),
                  loss='binary_crossentropy',
                  metrics=['accuracy', Recall()])

    return model

In [34]:
## set up early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [35]:
# Initialize Keras Tuner with val_recall as objective
tuner = kt.RandomSearch(
    build_model,
    objective=kt.Objective("val_recall", direction="max"),  # Maximizing Recall
    max_trials=5
)

# Use pre-split validation data
tuner.search(X_train, y_train, epochs=20, batch_size=10, validation_data=(X_val, y_val))

Reloading Tuner from ./untitled_project/tuner0.json


In [36]:
# Get best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"Best number of units: {best_hps.get('units')}")
print(f"Best number of layers: {best_hps.get('layers')}")
print(f"Best learning rate: {best_hps.get('learning_rate')}")

for i in range(best_hps.get('layers')):
  print(f"Best number of units in layer {i}: {best_hps.get(f'layer_{i}_units')}")

Best number of units: 64
Best number of layers: 2
Best learning rate: 0.001
Best number of units in layer 0: 64
Best number of units in layer 1: 128


In [37]:
# Train final model with best hyperparameters
best_model = tuner.hypermodel.build(best_hps)
best_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = best_model.fit(X_train, y_train, epochs=50,callbacks=early_stopping_callback , batch_size=10)


Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


800/800 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.8168 - loss: 0.4515
Epoch 2/50
 11/800 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.8361 - loss: 0.3924

/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


800/800 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8578 - loss: 0.3497
Epoch 3/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8534 - loss: 0.3502
Epoch 4/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.8602 - loss: 0.3313
Epoch 5/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8747 - loss: 0.3192
Epoch 6/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8699 - loss: 0.3230
Epoch 7/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8730 - loss: 0.3127
Epoch 8/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8728 - loss: 0.3067
Epoch 9/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8728 - loss: 0.3065
Epoch 10/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8716 - loss: 0.3067
Epoch 11/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8754 - loss: 0.2986
Epoch 12/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8739 - loss: 0.2948
Epoch 13/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accurac

In [38]:
best_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                      │ (None, 64)                  │             832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 40,325 (157.52 KB)

 Trainable params: 13,441 (52.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 26,884 (105.02 KB)

In [39]:
def evaluate_model(model, X, y):
  y_prob = model.predict(X)
  y_hat = (y_prob > 0.5).astype(int)

  Con_matrix = confusion_matrix(y, y_hat)
  print("Confusion Matrix")
  print(Con_matrix)
  print(f"Recall_score: {recall_score(y,y_hat)}")
  print(f"Precision_score: {precision_score(y,y_hat)}")
  print(f"F1 Score: {f1_score(y,y_hat)}")
  return

In [40]:
#for BASE MODEL
print("For train-data\n")
evaluate_model(model, X_train, y_train)
print("---------------------------------------------------------")
print("For val-data:\n")
evaluate_model(model, X_val, y_val)
print("---------------------------------------------------------")
print("For test-data:\n")
evaluate_model(model, X_test, y_test)

For train-data

250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Confusion Matrix
[[6174  182]
 [ 874  770]]
Recall_score: 0.4683698296836983
Precision_score: 0.8088235294117647
F1 Score: 0.5932203389830508
---------------------------------------------------------
For val-data:

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Confusion Matrix
[[751  34]
 [105 110]]
Recall_score: 0.5116279069767442
Precision_score: 0.7638888888888888
F1 Score: 0.6128133704735376
---------------------------------------------------------
For test-data:

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Confusion Matrix
[[796  26]
 [113  65]]
Recall_score: 0.3651685393258427
Precision_score: 0.7142857142857143
F1 Score: 0.483271375464684


In [41]:
# BEST MODEL
print("For train-data\n")
evaluate_model(best_model, X_train, y_train)
print("---------------------------------------------------------")
print("For val-data:\n")
evaluate_model(best_model, X_val, y_val)
print("---------------------------------------------------------")
print("For test-data:\n")
evaluate_model(best_model, X_test, y_test)


For train-data

250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Confusion Matrix
[[6231  125]
 [ 192 1452]]
Recall_score: 0.8832116788321168
Precision_score: 0.9207355738744452
F1 Score: 0.9015833592052158
---------------------------------------------------------
For val-data:

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Confusion Matrix
[[693  92]
 [ 96 119]]
Recall_score: 0.5534883720930233
Precision_score: 0.5639810426540285
F1 Score: 0.5586854460093896
---------------------------------------------------------
For test-data:

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Confusion Matrix
[[726  96]
 [105  73]]
Recall_score: 0.4101123595505618
Precision_score: 0.4319526627218935
F1 Score: 0.4207492795389049


In [43]:
y_prob = best_model.predict(X_test)

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


In [42]:
best_model.save('Model.h5')